In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((224, 224))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

 79%|████████████████████████████████████████████████████████████████▋                 | 15/19 [07:14<01:54, 28.56s/it]

In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\validation"

# Initialize empty lists for images and labels
X_val = []
Y_val = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((224, 224))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_val.append(img_array)
            Y_val.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print(f'X_train shape: {X_val.shape}')
print(f'Y_train shape: {Y_val.shape}')

In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\test"

# Initialize empty lists for images and labels
X_test = []
Y_test = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((224, 224))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_test.append(img_array)
            Y_test.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_test = np.array(X_test)
Y_test = np.array(Y_test)

print(f'X_train shape: {X_test.shape}')

print(f'Y_train shape: {Y_test.shape}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
tf.keras.backend.clear_session()
YT = to_categorical(Y_train)
VT = to_categorical(Y_val)

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet

num_classes = 19
# Create the ResNet model without the top (fully connected) layers
base_model = MobileNet(weights=None, include_top=False, input_shape=(224, 224, 3))    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
history = model.fit(X_train, YT, epochs=epochs, batch_size=8, validation_data=(X_val, VT), callbacks=[early_stopping])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(X_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

In [7]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
import keras

num_classes = 19
model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\mob_whoi_224_model.h5")

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)  # Explicitly specify keepdims
output = Dense(num_classes, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output) 
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
new_model.fit(X_train,YT, epochs=epochs,batch_size=8, validation_data=(X_val,VT))

Epoch 1/50
359/359 [==============================] - 28s 51ms/step - loss: 2.0036 - accuracy: 0.3907 - val_loss: 4.6335 - val_accuracy: 0.2581
Epoch 2/50
359/359 [==============================] - 17s 48ms/step - loss: 1.2338 - accuracy: 0.6166 - val_loss: 4.1413 - val_accuracy: 0.3871
Epoch 3/50
359/359 [==============================] - 17s 47ms/step - loss: 0.8248 - accuracy: 0.7413 - val_loss: 3.1214 - val_accuracy: 0.3871
Epoch 4/50
359/359 [==============================] - 16s 46ms/step - loss: 0.6555 - accuracy: 0.8040 - val_loss: 2.1879 - val_accuracy: 0.4839
Epoch 5/50
359/359 [==============================] - 16s 45ms/step - loss: 0.5018 - accuracy: 0.8499 - val_loss: 1.9223 - val_accuracy: 0.5591
Epoch 6/50
359/359 [==============================] - 17s 48ms/step - loss: 0.4113 - accuracy: 0.8722 - val_loss: 2.6745 - val_accuracy: 0.3763
Epoch 7/50
359/359 [==============================] - 18s 51ms/step - loss: 0.3647 - accuracy: 0.8858 - val_loss: 3.2346 - val_accuracy:

In [8]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = new_model.predict(X_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9915254237288136
F1 Score: 0.9915254237288136


In [10]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend
from keras import layers
from keras import utils
import keras
import tensorflow

class LayerScale(layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tensorflow.Variable(
            self.init_values * tensorflow.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

num_classes = 19
model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\conv_whoi_224_model.h5",custom_objects={ "LayerScale": LayerScale })

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D(keepdims=False)(x)  # Explicitly specify keepdims
output = Dense(num_classes, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output) 
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
history = new_model.fit(X_train, YT, epochs=epochs, batch_size=8, validation_data=(X_val, VT), callbacks=[early_stopping])

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
359/359 [==============================] - 508s 1s/step - loss: 3.0394 - accuracy: 0.1835 - val_loss: 2.4462 - val_accuracy: 0.1075
Epoch 2/50
359/359 [==============================] - 551s 2s/step - loss: 2.3183 - accuracy: 0.2653 - val_loss: 1.8801 - val_accuracy: 0.2903
Epoch 3/50
359/359 [==============================] - 602s 2s/step - loss: 1.9772 - accuracy: 0.3524 - val_loss: 2.0454 - val_accuracy: 0.3226
Epoch 4/50
359/359 [==============================] - 678s 2s/step - loss: 1.6606 - accuracy: 0.4614 - val_loss: 1.8060 - val_accuracy: 0.3333
Epoch 5/50
359/359 [==============================] - 409s 1s/step - loss: 1.3273 - accuracy: 0.5790 - val_loss: 1.6974 - val_accuracy: 0.4409
Epoch 6/50
359/359 [==============================] - 401s 1s/step - loss: 1.0553 - accuracy: 0.6793 - val_loss: 1.5935 - val_accuracy: 0.4624
Epoch 7/50
359/359 [==============================] - 406s 1s/step - loss: 0.9091 - accuracy: 0.7159 - val_loss: 1.4694 - val_accuracy: 0.5591

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = new_model.predict(X_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub

model = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/convnext/frameworks/TensorFlow2/variations/base-1k-224/versions/1")
])

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D(keepdims=False)(x)  # Explicitly specify keepdims
output = Dense(num_classes, activation='softmax')(x)
new_model = tf.keras.models.Model(inputs=model.input, outputs=output) 
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
new_model.summary()


AssertionError: Duplicate registrations for type trackable_dict_wrapper

In [ ]:
fit(X_train,YT, epochs=epochs,batch_size=8, validation_data=(X_val,VT))

In [1]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = new_model.predict(X_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")